In [ ]:
# Author: Jessica Ma
# Date: April 9, 2018
# Purpose: to jackknife with a single entry 
# Has a single query rather than iterating through all the files

In [39]:
# imports and getting the target protein
import subprocess
from subprocess import PIPE
import math
from __future__ import division

prot = input("File name: ")  
params = prot[-7:]

File name: 'PHYT00142-p40q40'


In [40]:
# function to write to csv
def print_csv(name, param, ta, na, cs, inc, orig):
    file = open('resultsf1.csv', 'a+')
    precision = inc/(inc + na)
    recall = inc/orig
    f1 = (2*precision*recall)/(precision + recall)
    ppos = param[2:4]
    qcovs = param[5:7]
    file.write(",".join([name, ppos, qcovs, str(ta), str(na), str(cs), str(inc), str(f1), str(orig)]) + '\n')

In [41]:
# intialize some constants
AMP = '/projects/spruceup/scratch/psitchensis/Q903/annotation/amp/'
NAMES = '{0}round2/temp_files/tmp_files{1}/{2}'.format(AMP, params, prot)
TESTDB = '{0}sequences/AMP_test_set-8Mar18.fa'.format(AMP)
DB = '{0}alignment/MSAs/db.fa'.format(AMP)

In [42]:
# remove (at least) 10% of the constructing set
constr = int(subprocess.check_output("wc -l < " + NAMES, shell=True))
rmv = int(math.ceil(0.1*constr))

In [43]:
# make all subsets w/o replacement

for i in range(1, constr - rmv + 1): 
    remover = 'sed "{0}d" {1} |'.format(i, NAMES)
    sedder =' sed "{0}d" |'.format(i)
    line = remover + sedder * (rmv-1) + ' seqtk subseq {0} "-" | muscle -out thing.msa'.format(DB)
    
    subprocess.call(line, shell=True)
    subprocess.call("hmmbuild thing.hmm thing.msa", shell=True)
    subprocess.call("hmmsearch -E 1e-5 --noali --notextw --tblout thingsearch thing.hmm " + TESTDB, shell=True)
    
    res = open("thingsearch").read()

    # get key values
    tot_amp = res.count(';AMP')
    non_amp = res.count(';non_AMP')
    hmmRes = set()
    cons = set()
    with open("thingsearch", "r") as infile:
        for line in infile:
            if line[0] != "#":
                rec = line.strip("\n").split("\t")[0]
                seqid = rec.split(";")[0]
                hmmRes.add(seqid)
                
    with open("temp_files/tmp_files-p40q40/" + prot, "r") as outfile:
        for line in outfile:
            cons.add(line.strip("\n"))
            
    incl = len(hmmRes.intersection(cons))
    
    # call instance of writing to csv
    print_csv(prot, params, tot_amp, non_amp, constr-rmv , incl, constr)